In [ ]:
# Libaries
import cv2
import requests
import json
import numpy as np
import os
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from os import getenv, stat
from pprint import pprint
import IPython.display as ipd
from PIL import Image

from tkinter import * #창 띄우는 라이브러리
import tkinter
#win = Tk() # 창 생성
#win.geometry("1000x500")
#win.mainloop() # 창 실행


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --target=$my_path jdc
! pip install gTTS
!apt-get install python3-tk

In [ ]:
my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/MyDrive/Colab Notebooks/my_env', my_path) #파일 경로 설정하는 부분
sys.path.insert(0, my_path)

In [ ]:
from gtts import gTTS #음성으로 알려주는 것
from IPython.display import Audio #Import Audio method from IPython's Display Class
import IPython.display as ipd


# List Image Files to Choose From
!ls test_img

# Select Test Image
#bus_data =input("원하는 사진을 입력하세요 : ")
#test_image = (bus_data) #"/content/drive/MyDrive/DeepLearning/Bus_Numbers-master/test_img/bus_ext_6.jpg"

# Opening the File
#image = Image.open(test_image)
#plt.title("Visualisation of Image Selected")
#imshow(image)

# Select Test Image
test_image = "/content/drive/MyDrive/DeepLearning/Bus_Numbers-master/test_img/bus_ext_6.jpg" #경로지정하는 부분

# Opening the File
image = Image.open(test_image)
plt.title("Visualisation of Image Selected")
imshow(image)

In [ ]:
## Ensure that the Image is of the Acceptable Size
while stat(test_image).st_size > 4100000:
    image = Image.open(test_image)
    x = int(image.size[0]*0.9)
    y = int(image.size[1]*0.9)
    image = image.resize((x,y), Image.ANTIALIAS)
    test_image = "/tmp/resized.jpg"
    image.save(test_image)
    print("Lowered Size of Image by 19%")

In [ ]:
# Use the Custom Vision Docker File (Superior Speed and Cost)
# NOTE: Change the following docker folder if you are using a different docker folder name
! ./custom_vision_docker/run_docker.sh
with open(test_image, 'rb') as image_file:
    
    #custom_vision_response = requests.post("http://127.0.0.1/image", data=image_file, headers={"Content-Type": "application/octet-stream"} )
! docker container kill $(docker ps -q)

In [ ]:
# Import Test Image into Python
raw_image = Image.open(test_image)
width, height = raw_image.size

# Set Points for Cropped Image to Bounding Box
left = 200 # width*bounding_box['left']
right =  450 #left + width*bounding_box['width']
top = 200 # height*bounding_box['top']
bottom =  280 #top + height*bounding_box['height']

# Crop Image
bus_num_image = raw_image.crop((left, top, right, bottom))
plt.title("Image of Bus Number")
imshow(np.asarray(bus_num_image)) #Display the Image
plt.plot()

In [ ]:
ocr_crop_percentage = 0.5
# Crop Away ocr_crop_percentage of Left Side for OCR Reasons
width, height = bus_num_image.size
left = width * ocr_crop_percentage
right = width
top = 0
bottom = height

ocr_image = bus_num_image.crop((left, top, right, bottom))

# Resize Image with Interpolation if height too big (somehow OCR on Azure doesn't work too well with too sharp of bus numbers)
width, height = ocr_image.size
if height > 50:
    ocr_image = ocr_image.resize((int(width*50/height),50),Image.ANTIALIAS) # Ensure the aspect ratio doesn't change
plt.title("Cropped Image")
imshow(np.asarray(ocr_image)) #Display the Image

# Save OCR Ready Image
ocr_image_file = "ocr.png"
ocr_image.save(ocr_image_file)

# Send Image Open API

In [ ]:
# Define Functions to Send Image to OCR Space API 
# Please don't steal the API key, it's fine to use here but don't use it for other things (just get another)
def ocr_space_file(filename, overlay=False, api_key='5fbfe9786f88957', language='eng'):
    payload = {'isOverlayRequired': overlay,
               'apikey': api_key,
               'language': language,
               'OCREngine': 2
               }
    with open(filename, 'rb') as f:
        r = requests.post('https://api.ocr.space/parse/image',
                          files={filename: f},
                          data=payload,
                          )
    return r.content.decode()


def parse_image(img_name):
    response = ocr_space_file(img_name)
    result = json.loads(response)
    pprint(result)
    text = result["ParsedResults"][0]["ParsedText"]
    text = text.split("\n")[0]
    return text

In [ ]:
# Call the API and Extract Bus Number
try:
    bus_number = parse_image("resized.png")
except Exception as e:
    print(e)
    bus_number = ""
    
done = False
if bus_number != "":
    print("Number Detected:", bus_number)
    done = True
else:
    bus_number = parse_image("ocr.png")

if bus_number != "" and not done:
    print("Number Detected 2:", bus_number)
elif not done:
    print("Cant Read Number on Bus")
    raise Exception("Couldnt Read Bus Number")

# 음성으로 말하기

In [ ]:
# Prepare Audio File
#audio_text = "Bus " + bus_number + " is coming now!"
audio_text = bus_number + "번 버스가 도착합니다"
language = 'ko'
#language = 'en' #'ko'로 바꾸면 한국어
audio_obj = gTTS(text=audio_text, lang=language, slow=False) 
audio_obj.save("bus_number.wav") 

# Display Option to Play File in Notebook
ipd.Audio('bus_number.wav')